## Add available processors

In [2]:
using Distributed
nprocs()

1

In [3]:
nprocs() == 1 && addprocs()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [4]:
workers()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

## Remote Calls and References

In [5]:
ref = @spawn sin(π/5)

Future(2, 1, 10, nothing)

In [6]:
fetch(ref)

0.5877852522924731

In [10]:
n=10
refs = Array{Future,1}(undef,n)
for i = 1:n
    refs[i] = @spawn sin(i)
end

In [11]:
refs

10-element Array{Future,1}:
 Future(5, 1, 22, nothing)
 Future(6, 1, 23, nothing)
 Future(7, 1, 24, nothing)
 Future(8, 1, 25, nothing)
 Future(9, 1, 26, nothing)
 Future(2, 1, 27, nothing)
 Future(3, 1, 28, nothing)
 Future(4, 1, 29, nothing)
 Future(5, 1, 30, nothing)
 Future(6, 1, 31, nothing)

In [12]:
reduce(+,map(fetch,refs))

1.4111883712180104

In [13]:
res=@distributed (vcat) for i=1:n
    sin(i)
end

10-element Array{Float64,1}:
  0.8414709848078965 
  0.9092974268256817 
  0.1411200080598672 
 -0.7568024953079282 
 -0.9589242746631385 
 -0.27941549819892586
  0.6569865987187891 
  0.9893582466233818 
  0.4121184852417566 
 -0.5440211108893698 

In [14]:
reduce(+,res)

1.4111883712180104

In [20]:
@everywhere using DataFrames
@sync res=@distributed (vcat) for i = 1:10
    println((i,sin(i)))
    DataFrame(worker=myid(), sin=sin(i))
end

      From worker 4:	(5, -0.9589242746631385)
      From worker 9:	(10, -0.5440211108893698)
      From worker 5:	(6, -0.27941549819892586)
      From worker 6:	(7, 0.6569865987187891)
      From worker 8:	(9, 0.4121184852417566)
      From worker 7:	(8, 0.9893582466233818)
      From worker 3:	(3, 0.1411200080598672)
      From worker 3:	(4, -0.7568024953079282)
      From worker 2:	(1, 0.8414709848078965)
      From worker 2:	(2, 0.9092974268256817)


,worker,sin
,Int64,Float64
1,2,0.841471
2,2,0.909297
3,3,0.14112
4,3,-0.756802
5,4,-0.958924
6,5,-0.279415
7,6,0.656987
8,7,0.989358
9,8,0.412118


In [21]:
sum(res[:,2])

1.4111883712180104

## Monte-Carlo Simulation to estimate $\pi$

In [22]:
#==========================#
# monte-carlo simulation
# π r^2 / 4 r^2 = s/n 
#==========================#


@everywhere function isInside() 
    x = rand()
    y = rand()
    x^2 + y^2 < 1 ? 1 : 0
end;

@everywhere function ppi(n)
    s=@distributed (+) for i = 1:n
        isInside()
    end
    4s/n
end;


In [23]:
@time ppi(10^9)

  2.055649 seconds (538.02 k allocations: 27.646 MiB, 0.53% gc time)


3.141542924

## Distributed Arrays

In [24]:
@everywhere using DistributedArrays

a = round.(10*rand(20,20))

20×20 Array{Float64,2}:
  3.0  4.0   5.0   3.0  2.0   9.0  2.0  …  0.0   6.0   8.0  10.0  1.0  3.0
  2.0  2.0   4.0   5.0  4.0   9.0  5.0     1.0   6.0   9.0   2.0  9.0  5.0
  0.0  2.0   7.0   3.0  2.0   4.0  3.0     9.0   9.0  10.0   7.0  5.0  4.0
  7.0  5.0  10.0   5.0  1.0   4.0  1.0     8.0   4.0   3.0   9.0  9.0  3.0
  0.0  8.0   9.0   8.0  7.0   9.0  1.0     4.0   2.0   7.0   2.0  3.0  8.0
  2.0  8.0   2.0   3.0  5.0   4.0  1.0  …  7.0   7.0   6.0   8.0  3.0  7.0
  8.0  7.0   5.0   4.0  6.0   8.0  8.0     3.0   1.0   1.0   2.0  2.0  6.0
  6.0  1.0   3.0   4.0  2.0   5.0  5.0     9.0   0.0   2.0   4.0  6.0  1.0
  4.0  6.0   3.0   4.0  2.0   3.0  5.0     7.0   3.0   5.0   5.0  2.0  1.0
  8.0  2.0   8.0   2.0  5.0   2.0  5.0     2.0   0.0   4.0   5.0  8.0  8.0
  7.0  1.0   5.0   3.0  8.0   8.0  6.0  …  4.0   6.0   3.0   3.0  3.0  8.0
  3.0  2.0   8.0   0.0  6.0   9.0  4.0     8.0  10.0   7.0   9.0  9.0  3.0
  9.0  7.0   6.0   3.0  8.0   9.0  8.0     4.0   3.0   1.0   7.0  2.0  9.0
 

In [25]:
map(x->myid(),a)

20×20 Array{Int64,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1 

In [26]:
da = distribute(a)

20×20 DArray{Float64,2,Array{Float64,2}}:
  3.0  4.0   5.0   3.0  2.0   9.0  2.0  …  0.0   6.0   8.0  10.0  1.0  3.0
  2.0  2.0   4.0   5.0  4.0   9.0  5.0     1.0   6.0   9.0   2.0  9.0  5.0
  0.0  2.0   7.0   3.0  2.0   4.0  3.0     9.0   9.0  10.0   7.0  5.0  4.0
  7.0  5.0  10.0   5.0  1.0   4.0  1.0     8.0   4.0   3.0   9.0  9.0  3.0
  0.0  8.0   9.0   8.0  7.0   9.0  1.0     4.0   2.0   7.0   2.0  3.0  8.0
  2.0  8.0   2.0   3.0  5.0   4.0  1.0  …  7.0   7.0   6.0   8.0  3.0  7.0
  8.0  7.0   5.0   4.0  6.0   8.0  8.0     3.0   1.0   1.0   2.0  2.0  6.0
  6.0  1.0   3.0   4.0  2.0   5.0  5.0     9.0   0.0   2.0   4.0  6.0  1.0
  4.0  6.0   3.0   4.0  2.0   3.0  5.0     7.0   3.0   5.0   5.0  2.0  1.0
  8.0  2.0   8.0   2.0  5.0   2.0  5.0     2.0   0.0   4.0   5.0  8.0  8.0
  7.0  1.0   5.0   3.0  8.0   8.0  6.0  …  4.0   6.0   3.0   3.0  3.0  8.0
  3.0  2.0   8.0   0.0  6.0   9.0  4.0     8.0  10.0   7.0   9.0  9.0  3.0
  9.0  7.0   6.0   3.0  8.0   9.0  8.0     4.0   3.0   1.0

In [27]:
map(x->myid(),da)

20×20 DArray{Int64,2,Array{Int64,2}}:
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 2  2  2  2  2  4  4  4  4  4  6  6  6  6  6  8  8  8  8  8
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3  3  3  3  3  5  5  5  5  5  7  7  7  7  7  9  9  9  9  9
 3

In [28]:
da.pids

2×4 Array{Int64,2}:
 2  4  6  8
 3  5  7  9

In [29]:
[@spawnat p sum(localpart(da)) for p in da.pids]

2×4 Array{Future,2}:
 Future(2, 1, 3618, nothing)  …  Future(8, 1, 3624, nothing)
 Future(3, 1, 3619, nothing)     Future(9, 1, 3625, nothing)

In [30]:
map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids])

2×4 Array{Float64,2}:
 218.0  235.0  253.0  241.0
 246.0  279.0  255.0  252.0

In [31]:
reduce(+,map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids]))

1979.0

In [32]:
sum(da)

1979.0

## Cross-validation in parallel

In [34]:
#using RDatasets
@everywhere using RDatasets
@everywhere using Statistics
#using DecisionTree
@everywhere using DecisionTree

@everywhere function irisAcc() 
    iris = dataset("datasets", "iris")
    features = iris[:, 1:4] |>Matrix;
    labels = iris[:, 5] |> Vector;
    model = build_forest(labels, features, 2, 4, 0.5, 6);
    accuracy = nfoldCV_forest(labels, features, 2, 4, 2, 0.5);
    mean(accuracy)
end

In [35]:
function mserial(n)
    sm=0.0
    for i=1:n
         sm += irisAcc()
    end
    print("Overall Acc:",sm/n*100.0)
end
@time mserial(100)


Fold 1
Classes:  

3×3 Array{Int64,2}:
 23   0   0
  0  29   0
  0   2  21

CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 

3×3 Array{Int64,2}:
 27   0   0
  0  18   3
  0   4  23

3×3 Array{Int64,2}:
 27   0   0
  0  22   0
  0   3  23

...

3×3 Array{Int64,2}:
 23   0   0
  0  23   1
  0   3  25

0.96
Kappa:    0.9390574214517876

Mean Accuracy: 0.9666666666666667

Fold 1
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.96
Kappa:    0.9393367484497168

Fold 2
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 1.0
Kappa:    1.0

Mean Accuracy: 0.98

Fold 1
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9733333333333334
Kappa:    0.9595687331536389

Fold 2
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9733333333333334
Kappa:    0.9596122778675283

Mean Accuracy: 0.9733333333333334

Fold 1
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.92
Kappa:    0.8793565683646113

Fold 2
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9866666666666667
Kappa:    0.9797023004059542

Mean Accuracy: 0.9533333333333334

Fol

3×3 Array{Int64,2}:
 24   0   0
  0  26   2
  0   1  22

3×3 Array{Int64,2}:
 26   0   0
  0  22   0
  0   1  26

0.9466666666666667
Kappa:    0.919871794871795

Mean Accuracy: 0.9666666666666667

Fold 1
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.96
Kappa:    0.9398556535685645

Fold 2
Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
Matrix:   
Accuracy: 0.9866666666666667
Kappa:    0.9799518845228548

Mean Accuracy: 0.9733333333333334
Overall Acc:96.5133333333333 22.915654 seconds (68.67 M allocations: 3.253 GiB, 9.55% gc time)


In [36]:
function mparallel(n)
    s=@distributed (+) for i=1:n
        irisAcc()
    end
    print("Overall Acc:",s/n*100.0)
end
@time mparallel(100)

      From worker 4:	
      From worker 3:	
      From worker 4:	Fold 1
      From worker 3:	Fold 1
      From worker 2:	
      From worker 2:	Fold 1
      From worker 3:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 4:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 2:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 5:	
      From worker 3:	Matrix:   3×3 Array{Int64,2}:
      From worker 4:	Matrix:   3×3 Array{Int64,2}:
      From worker 5:	Fold 1
      From worker 2:	Matrix:   3×3 Array{Int64,2}:
      From worker 3:	 29   0   0
      From worker 3:	  0  20   1
      From worker 3:	  0   0  25
      From worker 3:	Accuracy: 0.9866666666666667
      From worker 3:	Kappa:    0.9798061389337642
      From worker 4:	 26   0   0
      From worker 4:	  0  24   1
      From worker 4:	  0   0  24
      From worker 4:	Accuracy: 0.9866666666666667
      From worker 4:

      From worker 4:	Fold 1
      From worker 3:	 24   0   0
      From worker 4:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 4:	Matrix:   3×3 Array{Int64,2}:
      From worker 4:	 25   0   0
      From worker 4:	  0  22   2
      From worker 4:	  0   0  26
      From worker 4:	Accuracy: 0.9733333333333334
      From worker 4:	Kappa:    0.9599358974358975
      From worker 3:	  0  21   1
      From worker 3:	  0   1  28
      From worker 3:	Accuracy: 0.9733333333333334
      From worker 3:	Kappa:    0.9597207303974222
      From worker 5:	
      From worker 5:	Fold 1
      From worker 5:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 5:	Matrix:   3×3 Array{Int64,2}:
      From worker 5:	 26   0   0
      From worker 5:	  0  24   0
      From worker 5:	  0   0  25
      From worker 5:	Accuracy: 1.0
      From worker 5:	Kappa:    1.0
      From worker 3:	
      From worker 3:	Fold 2
      From worker

      From worker 4:	  0  25   4
      From worker 4:	  0   1  22
      From worker 4:	Accuracy: 0.9333333333333333
      From worker 4:	Kappa:    0.8998397435897436
      From worker 4:	
      From worker 4:	Mean Accuracy: 0.9533333333333334
      From worker 2:	Matrix:   3×3 Array{Int64,2}:
      From worker 2:	 27   0   0
      From worker 2:	  0  21   1
      From worker 2:	  0   0  26
      From worker 2:	Accuracy: 0.9866666666666667
      From worker 2:	Kappa:    0.979903536977492
      From worker 5:	
      From worker 5:	Fold 2
      From worker 2:	
      From worker 2:	Fold 2
      From worker 2:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 2:	Matrix:   3×3 Array{Int64,2}:
      From worker 2:	 23   0   0
      From worker 2:	  0  27   1
      From worker 2:	  0   2  22
      From worker 2:	Accuracy: 0.96
      From worker 2:	Kappa:    0.9397106109324759
      From worker 2:	
      From worker 2:	Mean Accuracy: 0.9733333333333334
  

      From worker 7:	
      From worker 8:	
      From worker 9:	
      From worker 7:	Fold 1
      From worker 8:	Fold 1
      From worker 9:	Fold 1
      From worker 6:	
      From worker 6:	Fold 1
      From worker 8:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 9:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 7:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 6:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 9:	Matrix:   3×3 Array{Int64,2}:
      From worker 8:	Matrix:   3×3 Array{Int64,2}:
      From worker 7:	Matrix:   3×3 Array{Int64,2}:
      From worker 8:	 24   0   0
      From worker 8:	  0  24   3
      From worker 8:	  0   0  24
      From worker 8:	Accuracy: 0.96
      From worker 8:	Kappa:    0.9400479616306954
      From worker 8:	
      From worker 8:	Fold 2
      From worker 8:	Classes:  Categoric

      From worker 7:	Mean Accuracy: 0.9733333333333334
      From worker 8:	
      From worker 8:	Fold 2
      From worker 8:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 8:	Matrix:   3×3 Array{Int64,2}:
      From worker 8:	 29   0   0
      From worker 8:	  0  20   4
      From worker 8:	  0   0  22
      From worker 8:	Accuracy: 0.9466666666666667
      From worker 8:	Kappa:    0.9196141479099679
      From worker 9:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 8:	
      From worker 8:	Mean Accuracy: 0.9666666666666667
      From worker 9:	Matrix:   3×3 Array{Int64,2}:
      From worker 9:	 28   0   0
      From worker 9:	  0  24   1
      From worker 9:	  0   0  22
      From worker 9:	Accuracy: 0.9866666666666667
      From worker 9:	Kappa:    0.9799196787148596
      From worker 9:	
      From worker 9:	Mean Accuracy: 0.98
      From worker 7:	
      From worker 7:	Fold 1
      From worker 7

      From worker 7:	
      From worker 7:	Fold 1
      From worker 7:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 7:	Matrix:   3×3 Array{Int64,2}:
      From worker 7:	 27   0   0
      From worker 7:	  0  24   2
      From worker 8:	
      From worker 9:	Matrix:   3×3 Array{Int64,2}:
      From worker 9:	 29   0   0
      From worker 9:	  0  21   1
      From worker 9:	  0   1  23
      From worker 9:	Accuracy: 0.9733333333333334
      From worker 9:	Kappa:    0.9597207303974222
      From worker 9:	
      From worker 9:	Mean Accuracy: 0.9666666666666667
      From worker 7:	  0   1  21
      From worker 7:	Accuracy: 0.96
      From worker 7:	Kappa:    0.9398395721925134
      From worker 8:	Fold 2
      From worker 8:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 8:	Matrix:   3×3 Array{Int64,2}:
      From worker 8:	 27   0   0
      From worker 8:	  0  19   2
      From worker 8:	  0   0  27
 

      From worker 6:	
      From worker 6:	Fold 1
      From worker 6:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 6:	Matrix:   3×3 Array{Int64,2}:
      From worker 6:	 29   0   0
      From worker 6:	  0  24   0
      From worker 6:	  0   2  20
      From worker 6:	Accuracy: 0.9733333333333334
      From worker 6:	Kappa:    0.9596774193548387
      From worker 6:	
      From worker 6:	Fold 2
      From worker 6:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      From worker 6:	Matrix:   3×3 Array{Int64,2}:
      From worker 6:	 21   0   0
      From worker 6:	  0  25   1
      From worker 6:	  0   2  26
      From worker 6:	Accuracy: 0.96
      From worker 6:	Kappa:    0.9396135265700483
      From worker 6:	
      From worker 6:	Mean Accuracy: 0.9666666666666667
      From worker 6:	
      From worker 6:	Fold 1
      From worker 6:	Classes:  CategoricalString{UInt8}["setosa", "versicolor", "virginica"]
      Fr